# Пример использования библиотеки gensim для тематического моделирования

Такая полезная теорема Байеса! :)

![comic1](http://imgs.xkcd.com/comics/seashell.png)

In [8]:
from gensim import corpora, models

 lda, действительно, очень популярны, и часто их используют просто как «чёрный ящик», подают матрице частоты слов и получают на выходе построенные матрицы. Причина состоит в том, что хорошо понять, как обучается lda, можно только, прослушав курс байесовских методов машинного обучения. 

In [9]:
# Импортируем данные в формте UCI Bag of Words
data = corpora.UciCorpus("docword.xkcd.txt", "vocab.xkcd.txt")
dictionary = data.create_dictionary()

 импортируем компоненты corpora и models из модуля gensim. Первый поможет нам импортировать данные, а второй — построить непосредственно модель. Данные у нас представлены в формате UCI Bag of Words. Это популярный формат, который позволяет максимально сжато представить разреженную матрицу частот слов. Давайте посмотрим пример. Docword.xkcd.txt-Это исходные данные. Здесь указаны ссылки на комиксы и текстовые описания того, что на них изображено. Из этих текстов удалена вся пунктуация и выполнена лимитизация, то есть приведение каждого слова к начальной форме. В итоге я получена матрица частот слов. В этом файле в первой строке записано количество документов, во второй — количество слов, в третьей — общее количество слов во всех документах коллекции. Дальше в каждой строке идёт три числа — это номер документа, номер слова и количество, сколько раз это слово встретилось в документе. Нумерация и документов, и слов здесь ведётся с единицы, а не с нуля как в Python.Тексты здесь во втором файле docword.xkcd.txt, который обязательно должен идти рядом с первым. Это файл-словарь. Здесь в каждой строке с номером I записано слово, которое отвечает этому индексу. То есть, например, под индексом 1 у нас идёт слово boy. Итак, давайте импортируем эти файлы в нашу модель. Для этого мы создадим объект класса UciCorpus и укажем путь к двум нашим файлам.Также gensim требует, чтобы мы в отдельную переменную обязательно сохранили словарь. Это можно сделать с помощью метода create_dictionary у объекта data, который мы получили в первой строке.

In [10]:
# обучение модель
%time ldamodel = models.ldamodel.LdaModel(data, id2word=dictionary, num_topics=5, passes=20, alpha=1.25, eta=1.25)

CPU times: user 2min 8s, sys: 312 ms, total: 2min 9s
Wall time: 2min 9s


Oбучению модели:вызваем функцию ldamodel из компоненты models. Давайте посмотрим на её параметры. Первый параметр — это corpus, у нас это, он хранится в переменной data, мы её и указываем. Второй параметр, который, конечно, необязателен, но крайне желательно его указать, это id2word. Это, собственно, отображение индексов слов непосредственно в слова. У нас это как раз тот dictionary, который мы создали во второй строке предыдущей ячейки. Далее идут некоторые параметры, которые влияют на обучение модели. Если у вас коллекция небольшая, как в нашем случае, то можно сделать много проходов по коллекции, и тогда модель получится наиболее адекватной. Тогда нужно указать параметр passes большим, например 20. Если же у вас коллекция действительно большая, например, как «википедия», то делать по ней двадцать проходов будет очень долго. Тогда можно указывать один или два прохода, но нужно обязательно подобрать аккуратно другие параметры, влияющие на сходимость алгоритмов. Особенно это параметры decay и offset, но об этом мы не будем подробно говорить в нашей демонстрации. Кроме того, полезно указывать параметр distributed, например, в true, тогда вы сможете обучать модель сразу в нескольких параллельных процессах. Параметра модели — это alpha и eta. Они создают параметры априорного распределения Дирихле для моделей. Распределение Дирихле — это распределение над векторами, и параметр для него — это тоже вектор. Если в этом векторе параметра все числа одинаковые, то это симметричные распределения Дирихле, если разные — то асимметричные. Gensim поддерживает разные варианты. Вы можете подавать одно число, тогда оно будет... gensim повторит его несколько раз, оставит вектор, и это будет симметричный prayer. 
На практике достаточно попробывать числа с каким-то шагом в интервале от 0 до 2, и выбрать те, при которых модель получается наиболее интерпретируемой.

In [15]:
# Сохранение модели
ldamodel.save("ldamodel_xkcd")

In [16]:
# Загрузка модели
ldamodel = models.ldamodel.LdaModel.load("ldamodel_xkcd")

Для сохранения и загрузки моделей в gensim есть соответствующие функции. Первая — это ldamodel.save, здесь мы должны указать имя файла, куда сохранить модель. . И также это метод LdaModel.load, в который предварительно сохранена модель.

In [18]:
# выводим топы слов
for t, top_words in ldamodel.print_topics(num_topics=10, num_words=10):
    print ("Topic", t, ":", top_words)

Topic 0 : 0.026*"b'man'" + 0.012*"b'text'" + 0.010*"b'woman'" + 0.010*"b'title'" + 0.009*"b'guy'" + 0.006*"b'girl'" + 0.006*"b'one'" + 0.005*"b'just'" + 0.005*"b'hat'" + 0.005*"b'two'"
Topic 1 : 0.002*"b'paul'" + 0.002*"b'ron'" + 0.001*"b'reporter'" + 0.001*"b'beef'" + 0.001*"b'bag'" + 0.001*"b'unallocated'" + 0.001*"b'base'" + 0.001*"b'blimp'" + 0.001*"b'shark'" + 0.001*"b'destroy'"
Topic 2 : 0.005*"b'figure'" + 0.004*"b'person'" + 0.002*"b'text'" + 0.002*"b'title'" + 0.002*"b'one'" + 0.002*"b'people'" + 0.002*"b'teacher'" + 0.002*"b'part'" + 0.001*"b'stick'" + 0.001*"b'randall'"
Topic 3 : 0.021*"b'person'" + 0.004*"b'text'" + 0.004*"b'labeled'" + 0.004*"b'title'" + 0.003*"b'line'" + 0.003*"b'time'" + 0.002*"b'graph'" + 0.002*"b'one'" + 0.002*"b'number'" + 0.002*"b'map'"
Topic 4 : 0.002*"b'wait'" + 0.002*"b'sagal'" + 0.002*"b'peter'" + 0.001*"b'han'" + 0.001*"b'leopard'" + 0.001*"b'dont'" + 0.001*"b'turtle'" + 0.001*"b'solo'" + 0.001*"b'leia'" + 0.000*"b'city'"


Давайте загрузим готовую модель. Отдельно стоит обратить внимание на то, что, даже если вы уже зафиксировали все параметры, построенная модель будет очень зависеть от начального приближения, потому что изначально все параметры в модели инициализируются какими-то случайными числами. Поэтому каждый раз, если вам позволяет размер коллекции и ваше время, стоит построить модель несколько раз и выбрать ту, в которой темы лучше всего

Понять, какие темы хорошие или плохие можно, посмотрев на «топы» слов. Их можно вывести, вызвав функцию print_topics, указав, сколько тем вы хотите посмотреть и сколько слов вывести в каждой теме. Лексика в комиксах xkcd достаточно специфичная, и особенно русскоговорящему человеку не всегда понятная. Поэтому давайте скопируем наши темы в яндекс-переводчик и посмотрим переводы этих слов. Мы видим, что в первых двух темах много имён, и, честно сказать, не очень понятно, о чём эти темы. Третья тема, она посвящена комиксам в таком стиле: там любят рисовать график, и этот график как-то связан со временем, и идёт какой-то сопровождающий текст. Потом говорят слова: «рисунок», «метка», «линия», «год», «время». Третья тема — это тема для комиксов из серии, когда на нём есть мужчина, женщина, то есть два человека, и они что-то обсуждают. А четвёртая тема весьма специфична. Она рассказывает о каких-то... о выживании на острове, я бы сказала. Это слова: «ждать», «остров», «карта», «обнаружена», и какие-то сопутствующие слова

In [21]:
# Вычисляем логарифм перплексии и немного преобразуем, чтобы привести к общепринятому виду
perplexity = ldamodel.log_perplexity(list(data))
print (2**(-perplexity))

355.184310063


Перплексия — это очень распространённая мера качества тематических моделей, и особенное значение она имеет в контексте байесовских моделей. В первую очередь, потому что она описывает, насколько хорошо ваше распределение, описывает ваши исходные данные. И интерпретировать его можно так: чем меньше значение перплексии, тем лучше. С другой стороны, у перплексии нет максимального значения. То есть непонятно, если нам вручили число что оно означает. Величина перплексии зависит, во-первых, от данных, а во-вторых, от количества тем. Поэтому, во-первых, нельзя сравнивать перплексию модели, построенных на разных данных, а во-вторых, нельзя по перплексии выбирать количество тем. Это очень логично. Чем больше у нас тем, тем лучше наша матрица аппроксимирует исходную тему, то есть описывает её лучше, тем, соответственно, меньше перплексия. Иногда перплексию измеряют на тестовом dataset, то есть отдельно отложенных документах или частях документов, но это важно для научных статей, а для практики не очень важно, поэтому мы можем измерять перплексию прямо по обучающей выборке. Давайте это и сделаем. Сейчас мы убедимся в том, что перплексия, как само число, мало о чём говорит, то есть её можно использовать только для сравнения моделей, но нельзя понять, хорошая модель или плохая, только для одной модели. Ну собственно, так и получа

In [26]:
perp = ldamodel.bound(data)
2**(-perp/float(87409))

355.18431001294022

In [28]:
# Добавление в модель новых документов, содержащихся в новом корупсе data2
ldamodel.update(data2, passes=10)

функция update, которая позволяет дообучить модель на новых данных. То есть, если, например, вы бы считали ещё один другой corpus, например, из новых комиксов, в переменную data2 и указали бы её как первый аргумент функции update и также указали бы, сколько сделать проходов по коллекции, чтобы дообучить модель, то gensim обновит все параметры

In [25]:
# Получение распределения тем для конкретного документа
doc = list(data)[0]
ldamodel.get_document_topics(doc)

[(0, 0.75676409009022449),
 (1, 0.058410561277228691),
 (2, 0.057730747142185998),
 (3, 0.070441437964436124),
 (4, 0.056653163525924886)]

Ещё одна важная функция — это get_document_topics. Он позволяет для каждого документа найти распределение над множеством тем. Давайте получим распределение для другого документа и увидим, что в нём наибольшую роль играет третья тема. Напомним, что это тема про мужчин и женщин. И также мы увидим, что все числа в правом столбце суммируются к единице, что неудивительно, потому что в столбце матрицы тета — это дискретные вероятностные распределения. 

Эти люди не знают про тематические модели:

![comic2](http://imgs.xkcd.com/comics/the_problem_with_wikipedia.png) | ![comic3](http://imgs.xkcd.com/comics/mystery_news.png)

In [29]:
1+1

2

In [30]:
1-1

0